# Función para extraer las coordenadas de xml y exporta a csv

In [ ]:
from google.colab import drive
from google.colab import files
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  

import xml.etree.ElementTree as ET
import csv
import pandas 
import os

Not connected to a GPU


In [ ]:
#Aceso drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

DATASET_DIR = "/content/drive/MyDrive/Tomato_Weed_Detection/almacen"
ANNOTATIONS_FILE = "/content/drive/MyDrive/Tomato_Weed_Detection/anotations.csv" 
CLASSES_FILE = "/content/drive/MyDrive/Tomato_Weed_Detection/clases.csv" 

In [ ]:
annotations = []
classes = set([])
for roots, dirs, file in os.walk(DATASET_DIR, topdown=False):
  for xml_file in [f for f in os.listdir(roots) if f.endswith(".xml")]:
    tree = ET.parse(os.path.join(roots, xml_file))
    root = tree.getroot()
    file_name = None
    for elem in root:
      if elem.tag == 'filename':
        file_name = os.path.join(roots, elem.text)

      if elem.tag == 'object':
        obj_name = None
        coords = []
        for subelem in elem:
          if subelem.tag == 'name':
            obj_name = subelem.text
          if subelem.tag == 'bndbox':
            xmin = ""
            xmax = ""
            ymin = ""
            ymax = ""
            for subsubelem in subelem:
              f = float(subsubelem.text)
              i = int(f)
              if subsubelem.tag == "xmin":
                xmin = str(i)
              if subsubelem.tag == "xmax":
                xmax = str(i)
              if subsubelem.tag == "ymin":
                ymin = str(i)
              if subsubelem.tag == "ymax":
                ymax = str(i)
            
            coords.append(xmin)
            coords.append(ymin)
            coords.append(xmax)
            coords.append(ymax)
              
        item = [file_name] + coords + [obj_name]
        annotations.append(item)
        classes.add(obj_name)


print('Etiquetas listass')

Etiquetas listass


In [ ]:
#write anotations
with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

In [ ]:
#write clases
with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))

In [ ]:
df = pandas.read_csv(ANNOTATIONS_FILE)
df.columns
df.iloc[:,5:6].value_counts()

LYPES    
SOLNI        1188
CYP_CYPRO     941
POROL         255
NCI           231
LYPES         165
ECHCG         153
SETIT          56
dtype: int64